# Plant Disease Recognition

In [1]:
import ivy

In [2]:
ivy.set_backend("torch")

<module 'ivy.functional.backends.torch' from '/home/chaitanya/other/ivy/ivy/functional/backends/torch/__init__.py'>

In [ ]:
class PlantDiseaseRecogniser(ivy.Module):
    "Plant Disease Recognition model for 256x256 images."
    
    def __init__(self, num_classes: int):
        kernal_size = (3, 3)
        self.cnn = ivy.Sequential(
            ivy.Conv2D(3, 8, kernal_size, 1, "same"),
            ivy.BatchNorm2D(8),
            ivy.LeakyReLU(0.2),
            ivy.MaxPool2D((2,2), 2, 0),
            ivy.Conv2D(8, 32, kernal_size, 1, "same"),
            ivy.BatchNorm2D(32),
            ivy.LeakyReLU(0.2),
            ivy.MaxPool2D((2,2), 2, 0),
            ivy.Conv2D(32, 64, kernal_size, 1, "same"),
            ivy.BatchNorm2D(64),
            ivy.LeakyReLU(0.2),
            ivy.MaxPool2D((2,2), 2, 0),
            ivy.Conv2D(64, 128, kernal_size, 1, "same"),
            ivy.BatchNorm2D(128),
            ivy.LeakyReLU(0.2),
            ivy.MaxPool2D((2,2), 2, 0),
            ivy.Conv2D(128, 128, kernal_size, 1, "same"),
            ivy.BatchNorm2D(128),
            ivy.LeakyReLU(0.2),
        )

        self.fc = ivy.Sequential(
            ivy.Linear(32768, 1000),
            ivy.LeakyReLU(0.2),
            ivy.Dropout(0.2),
            ivy.Linear(1000, num_classes),
        )

        self.output_actv = ivy.Softmax()
        self.loss = ivy.CrossEntropyLoss()
        self.optimizer = ivy.Adam()
            
        super().__init__()